Pytorch

Working with Data

PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.


In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce GTX 1050 Ti
Using device: cuda


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

#create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# creating Models

In [5]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [6]:
# define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# optimizing the Model Parameters
# To train a model, we need a loss function and an optimizer.

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
# In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
# We also check the model’s performance against the test dataset to ensure it is learning.

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [14]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.107929  [   64/60000]
loss: 1.045792  [ 6464/60000]
loss: 0.946625  [12864/60000]
loss: 1.087713  [19264/60000]
loss: 1.166425  [25664/60000]
loss: 1.052570  [32064/60000]
loss: 1.073726  [38464/60000]
loss: 0.901367  [44864/60000]
loss: 1.022231  [51264/60000]
loss: 0.974303  [57664/60000]
Test Error: 
 Accuracy: 65.2%, Avg loss: 0.995949 

Epoch 2
-------------------------------
loss: 1.090110  [   64/60000]
loss: 0.978933  [ 6464/60000]
loss: 0.947284  [12864/60000]
loss: 1.129496  [19264/60000]
loss: 0.874004  [25664/60000]
loss: 0.825330  [32064/60000]
loss: 0.988245  [38464/60000]
loss: 0.951013  [44864/60000]
loss: 1.025639  [51264/60000]
loss: 0.897123  [57664/60000]
Test Error: 
 Accuracy: 67.4%, Avg loss: 0.923347 

Epoch 3
-------------------------------
loss: 1.092561  [   64/60000]
loss: 0.881014  [ 6464/60000]
loss: 0.974063  [12864/60000]
loss: 0.951928  [19264/60000]
loss: 1.049029  [25664/60000]
loss: 0.797995  [32064/600

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [16]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [30]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[2][0], test_data[2][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"


In [31]:
# predict all text data
classes = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot",
]

model.eval()  # Set model to evaluation mode
predictions = []
actuals = []

from torch.utils.data import DataLoader

test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        pred = model(X)  # Shape: [batch_size, num_classes]
        predicted_indices = pred.argmax(dim=1)  # Index of max probability

        # Convert indices to class names
        predictions.extend([classes[i] for i in predicted_indices.cpu().numpy()])
        actuals.extend([classes[i] for i in y.cpu().numpy()])


In [32]:
for i in range(10):
    print(f"Predicted: {predictions[i]}, Actual: {actuals[i]}")

Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Trouser, Actual: Trouser
Predicted: Trouser, Actual: Trouser
Predicted: Shirt, Actual: Shirt
Predicted: Trouser, Actual: Trouser
Predicted: Shirt, Actual: Coat
Predicted: Shirt, Actual: Shirt
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker


In [33]:
correct = sum([p == a for p, a in zip(predictions, actuals)])
accuracy = correct / len(test_data) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 75.87%
